In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


#### Import the data from the folders created to hold the original images and their corresponding image with a watermark. Standardize and reshape the images to fit into the network. 

In [3]:
raw_dir = 'C:/Users/kasra/Desktop/Cup/2/dataset/train/negative'
wat_dir = 'C:/Users/kasra/Desktop/Cup/2/dataset/train/positive'

raw_datagen = ImageDataGenerator(rescale=1./255)

raw_generator = raw_datagen.flow_from_directory(
        raw_dir,  # this is the target directory
        target_size=(250, 250),
        batch_size=5096, shuffle = False,
        class_mode="categorical")

wat_datagen = ImageDataGenerator(rescale=1./255)

wat_generator = wat_datagen.flow_from_directory(
        wat_dir,  # this is the target directory
        target_size=(250, 250),
        batch_size=5096, shuffle = 0,
        class_mode="categorical")

x_raw,y = raw_generator.next()
x_wat,y = wat_generator.next()

Found 4291 images belonging to 1 classes.
Found 4291 images belonging to 1 classes.


In [4]:
test_dir = 'C:/Users/kasra/Desktop/Cup/2/dataset/test'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=(250, 250),
        batch_size=1103, shuffle = False,
        class_mode="categorical")

x_test,y_test = test_generator.next()

Found 1103 images belonging to 1 classes.


In [5]:
x_raw.shape

(4291, 250, 250, 3)

#### Split the data into training and validation sets. 

In [6]:
x_train_raw = x_raw[0:200]
x_train_wat = x_wat[0:200]

x_val_raw = x_raw[200:250]
x_val_wat = x_wat[200:250]

x_train = np.vstack((x_train_raw, x_train_wat))
x_val = np.vstack((x_val_raw, x_val_wat))

In [7]:
y_train = np.repeat([0.], 200)
y_train = np.append(y_train, np.repeat([1.], 200))

y_val = np.repeat([0.], 50)
y_val = np.append(y_val, np.repeat([1.], 50))

In [8]:
x_train.shape

(800, 250, 250, 3)

#### Create the response variables for each data set: images with a watermark have a response of 1, while images without a watermark have a response of 0. 

In [173]:
model = keras.models.Sequential()
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='elu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='elu'))
model.add(layers.Conv2D(100, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((4, 4)))

model.add(layers.Conv2D(75, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(202, activation='elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(40, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['acc'])
#model.summary()
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 12, 
          batch_size = 25, verbose = 0)

In [ ]:
# make a prediction
yhat = model.predict(x_test[0:])

In [1]:
yhat

NameError: name 'yhat' is not defined

In [176]:
history.history

{'loss': [0.833861431106925,
  0.7269358411431313,
  0.7108206525444984,
  0.684205686673522,
  0.6834716014564037,
  0.6433127103373408,
  0.6230340581387281,
  0.6053322069346905,
  0.5718950042501092,
  0.5327417440712452,
  0.43991149589419365,
  0.43052427656948566],
 'acc': [0.51125,
  0.50375,
  0.52375,
  0.5525,
  0.57625,
  0.655,
  0.65375,
  0.68125,
  0.6725,
  0.73875,
  0.79125,
  0.8075],
 'val_loss': [0.6784938126802444,
  0.7163877189159393,
  0.6657157093286514,
  0.6721587628126144,
  0.6005875021219254,
  0.6310412362217903,
  0.5681379362940788,
  0.5793224424123764,
  0.6009222939610481,
  0.5103896260261536,
  0.43254416808485985,
  0.515699714422226],
 'val_acc': [0.56,
  0.53,
  0.6,
  0.58,
  0.71,
  0.67,
  0.73,
  0.69,
  0.68,
  0.75,
  0.8,
  0.77]}